# Diffractometer "Parameters"

Some of the diffractometer _modes_ use additional parameters.  The [E4CV](https://people.debian.org/~picca/hkl/hkl.html#org7ef08ba) geometry, for example, has a `double_diffraction` mode which requires a reference $hkl_2$ vector.  The vector is set and accessed by a Python command that calls directly the corresponding *libhkl* function:

action | `E4CV` method
:--- | :---
read names | `e4cv.calc.engine._engine.parameters_names_get(units_code)`
read values | `e4cv.calc.engine._engine.parameters_values_get(units_code)`
write names | Names are pre-defined by *libhkl*.
write values | `e4cv.calc.engine._engine.parameters_values_set(values, units_code)`

In [1]:
import gi
gi.require_version('Hkl', '5.0')
from hkl.calc import A_KEV

## `E4CV`, `hkl`, `double_diffraction`

term | value
:--- | :---
geometry | `E4CV`
engine | `hkl`
mode | `double_diffraction`

In [2]:
from hkl.geometries import SimulatedE4CV
e4cv = SimulatedE4CV("", name="e4cv")

# order:  a b c  alpha beta gamma
lattice = (3.781726143, 3.791444574, 3.79890313, 90.2546203, 90.01815424, 89.89967858)
e4cv.calc.new_sample("LNO_LAO", lattice=lattice)

wavelength = 1.239424258
e4cv.energy.put(A_KEV / wavelength)

# order:   omega    chi      phi  tth
p_002 = (19.1335, 90.0135, 0.0, 38.09875)
r_002 = e4cv.calc.sample.add_reflection(0, 0, 2, p_002)

p_113 = (32.82125, 115.23625, 48.1315, 65.644)
r_113 = e4cv.calc.sample.add_reflection(1, 1, 3, p_113)

e4cv.calc.sample.compute_UB(r_002, r_113)

array([[-9.55499011e-02, -1.65427863e+00,  2.42844485e-03],
       [ 2.62981975e-04,  9.81483906e-03,  1.65396181e+00],
       [-1.65871244e+00,  9.82002396e-02, -3.89705577e-04]])

In [3]:
print(f"{e4cv.calc.engine.mode = }")
e4cv.calc.engine.mode = "double_diffraction"
print(f"{e4cv.calc.engine.mode = }")

e4cv.calc.engine.mode = 'bissector'
e4cv.calc.engine.mode = 'double_diffraction'


Need to set parameters `h2`, `k2`, `l2`.  How?

First, there is a shorcut *property*.  It simplifies a more complicated to the lower level support.

In [8]:
e4cv.calc.parameters

['h2', 'k2', 'l2']

The lower-level call looks like this:

In [10]:
e4cv.calc.engine._engine.parameters_names_get()

['h2', 'k2', 'l2']

Read the parameters.  The value is 0 or 1, pick 1 for user units. (*libhkl*: `Hkl.UnitEnum.USER` == 1)

In [5]:
e4cv.calc.engine._engine.parameters_values_get(1)

[1.0, 1.0, 1.0]

Set the parameters.  First are the values, then the 0 or 1.

In [6]:
# no shortcut to this one.  Yet.
e4cv.calc.engine._engine.parameters_values_set((2,2,0), 1)

1